# A Simple Network in Keras with Callbacks

TensorFlow provides callbacks that can be used to extend the functionality of the training loop.

In this notebook we explore a few examples.

We have previously seen how we can store metrics such as the loss function or the accuracy in
the history object returned by the ```fit()``` method.
TensorFlow provides a much more detailed system for monitoring and analyzing training sessions
as well as model performance which is called (TensorBoard)[https://www.tensorflow.org/tensorboard].
To be able to use this functionality, the special "hooks" or functions have to be added
to the training loops that take care of logging the relevant information. This is done as a "callback".

Another useful callback to add are model checkpoints. Training complex neural networks may take a
long time and if for example the computer crashes during training, all training progress is lost.
However, we can add another callback to save intermiate trainning progress at regular intervals.
Should something happen, we can then resume the training from this point onwards.

In this example we use the [Iris Dataset](http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)
again with a simple network we have used earlier.

In [7]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix
import pydot

%pylab inline
# large figures
rcParams['figure.figsize'] = 8, 6

print(tf.__version__)

2.1.0


In [8]:
#load the data
iris = load_iris()

In [9]:
#define the network model
n_input = 4
n_output = 3
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(n_input,)))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(n_output))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_4 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 33        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [10]:
# setup the model
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
)

Now we setup the callbacks.
Note that the model checkpoints make the training quite slow as the checkpoints have to be written to disk. Hence we only make a checkpoint every few training iterations (i.e. period or epoch)

In [11]:
callbacks = [
    # write logs to ./logs
    tf.keras.callbacks.TensorBoard(log_dir='./logs/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))),
    # write checkpoints to ./checkpoints
    tf.keras.callbacks.ModelCheckpoint(filepath='./checkpoints/model{epoch:02d}.h5', save_freq='epoch'),
]

history = model.fit(
        x=iris.data,
        y=iris.target,
        batch_size=32,
        epochs=50,
        verbose=1,
        callbacks=callbacks
    )

Train on 150 samples
Epoch 1/50
150/150 [==============================] - 1s 3ms/sample - loss: 1.1109 - sparse_categorical_accuracy: 0.3800
Epoch 2/50
150/150 [==============================] - 0s 302us/sample - loss: 0.9754 - sparse_categorical_accuracy: 0.3533
Epoch 3/50
150/150 [==============================] - 0s 295us/sample - loss: 0.8148 - sparse_categorical_accuracy: 0.7200
Epoch 4/50
150/150 [==============================] - 0s 288us/sample - loss: 0.7024 - sparse_categorical_accuracy: 0.6733
Epoch 5/50
150/150 [==============================] - 0s 328us/sample - loss: 0.6043 - sparse_categorical_accuracy: 0.7733
Epoch 6/50
150/150 [==============================] - 0s 303us/sample - loss: 0.5217 - sparse_categorical_accuracy: 0.8867
Epoch 7/50
150/150 [==============================] - 0s 308us/sample - loss: 0.4542 - sparse_categorical_accuracy: 0.9067
Epoch 8/50
150/150 [==============================] - 0s 308us/sample - loss: 0.3970 - sparse_categorical_accuracy: 0.95

After training we can confirm that we have the model saved after each complete epoch
and we also have the relevant logs for TensorBoard.

In [12]:
!ls logs checkpoints


checkpoints:
model01.h5  model10.h5	model19.h5  model28.h5	model37.h5  model46.h5
model02.h5  model11.h5	model20.h5  model29.h5	model38.h5  model47.h5
model03.h5  model12.h5	model21.h5  model30.h5	model39.h5  model48.h5
model04.h5  model13.h5	model22.h5  model31.h5	model40.h5  model49.h5
model05.h5  model14.h5	model23.h5  model32.h5	model41.h5  model50.h5
model06.h5  model15.h5	model24.h5  model33.h5	model42.h5
model07.h5  model16.h5	model25.h5  model34.h5	model43.h5
model08.h5  model17.h5	model26.h5  model35.h5	model44.h5
model09.h5  model18.h5	model27.h5  model36.h5	model45.h5

logs:
2020-05-06-08-06-36  2020-05-06-12-39-28  2020-05-06-12-51-23
2020-05-06-08-15-49  2020-05-06-12-50-02


## Visualization with TensorBoard

Before, during or after trainig we can then open TensorBoard. 
From a commanline shell, we use ```tensorboard --log-dir logs``` where ```logs``` is the directory containing all the log-files we
have specified when setting up the TensorBoard callback.

We can then open a webbrowser and point it to the location where TensorBoard runs, e.g. ``` http://localhost:6006/```

An example is shown below:
![TensorBoard Example](TensorBoard_Example_Iris.png)

We can select the training runs we want to analyze or monitor live (in case the traning is progressing),
hover over the graphs with the mouse and obtain more details at a specific training step.